In [2]:
import pandas as pd
import numpy as np
import re

# Import Data
pro = pd.read_csv("product_details.csv")
cus = pd.read_csv("customer.csv")
ord = pd.read_csv("order_details.csv")

# Data Cleaning process

# 1. Remove unused columns
# 2. Check for null value
# 3. Check for duplicated value
# 4. For columns with limited value, 'Yes' and 'No' for instance, use df.unique() to check if the value mispelled or not
# 5. Adjust the column's type

In [3]:
# Cleaning
# 1. Product Data

# Check for null value and Drop columns with no value
pro2 = pro.iloc[:, [0,1,4,7,13,25]] # I create po2 cuz if i mistakenly change the content, i still have the origin data from pro to copy

# Deal with 'Category' column
# Create a new Table named Category with mulitiple categories display in one column only, and drop the column Category in the origin table
cat = pro2.assign(Categories=pro2['Category'].str.split(' \| ')).explode('Categories', ignore_index=True)
cat = cat[['Uniqe Id', 'Categories']]
cat = cat.rename(columns = {'Categories' : 'Category'})
cat['Category'] = cat['Category'].ffill()
pro2 = pro2.drop(['Category'], axis = 1)


# Deal with 'Selling Price' column
# Remove the $ symbol from Selling Price
pro2['Selling Price'] = pro2['Selling Price'].str.replace('$', '')

# Fill the 'Selling Price' null values with the above value
pro2['Selling Price'] = pro2['Selling Price'].ffill()

# Converts messily formatted cells to standard formatting (From '12.99 - 24.99','1,222.99','12 99', ... to 12.99)
pattern1 = r'^\d{1,3}\.\d{2}$'                                # 12.99 - standard formatting
pattern2 = r'^\d{1,3}\.\d{2} - \d{1,3}\.\d{2}$'               # 12.99 - 24.99
pattern3 = r'^\d{1,3}\,\d{1,3}\.\d{2}$'                       # 1,222.99
pattern4 = r'^\d{1,3}\.\d{2}  \d{1,3} \. \d{2}$'              # 12.99  12 . 99 
pattern5 = r'^Total price:$'                                  # Total Price
pattern6 = r'^\d{1,3}\.\d{2} - \d{1,3}\,\d{1,3}\.\d{2}$'      # 12.99 - 1,111.99
pattern7 = r'^\d{1,3}\.\d{2} \d{1,3}\.\d{2}$'                 # 12.99 12.99 
pattern8 = r'^ \d{1,3} \d{1,3}$'                              # 12 99 
pattern9 = r'^ \d{1,3} \d{1,3} \d{1,3}\.\d{2}.*'              # 19 99 39.95 #listPriceLegalMe ...
pattern10 = r'^\d{1,3}\.\d{1,3}  \d{1,3}$'                    # 8.99 8


for i, data in pro2['Selling Price'].items():
    if re.match(pattern1, data):
        pro2.loc[i, 'Selling Price'] = data
    elif re.match(pattern2, data):
        price_range = data.split('-')
        lower_bound = price_range[0]
        upper_bound = price_range[1]
        pro2.loc[i, 'Selling Price'] = (float(lower_bound) + float(upper_bound)) / 2
    elif re.match(pattern3, data):
        pro2.loc[i, 'Selling Price'] = data.replace(',', '')
    elif re.match(pattern4, data):
        values = data.split()
        pro2.loc[i, 'Selling Price'] = values[0]
    elif data == 'Total price:':
        pro2.loc[i, 'Selling Price'] = pro2.loc[i-1, 'Selling Price']
    elif re.match(pattern6, data):
        price_range = data.split('-')
        lower_bound = price_range[0]
        upper_bound = price_range[1].replace(',', '')
        pro2.loc[i, 'Selling Price'] = (float(lower_bound) + float(upper_bound)) / 2
    elif re.match(pattern7, data):
        pro2.loc[i, 'Selling Price'] = data.split(' ')[0]
    elif re.match(pattern8, data):
        pro2.loc[i, 'Selling Price'] = data.replace(' ', '',1).replace(' ', '.',2)
    elif re.match(pattern9, data):
        price_range = data.split()
        lower_bound = price_range[0] + '.' + price_range[1]
        upper_bound = price_range[2]
        pro2.loc[i, 'Selling Price'] = (float(lower_bound) + float(upper_bound)) / 2
    elif re.match(pattern10, data):
        pro2.loc[i, 'Selling Price'] = data.split()[0]
    else:
        pro2.loc[i, 'Selling Price'] = pro2.loc[i-1, 'Selling Price']


# Deal with 'Shipping Weight' colum
# Fill 'Shipping Weight' null value with the above value
pro2['Shipping Weight'] = pro2['Shipping Weight'].ffill()

# There is a index that has '.pounds' only so i replace with the above value
pro2.loc[1619, 'Shipping Weight'] = pro2.loc[1618, 'Shipping Weight']

# There are some index with '1,88' instead of '1.88', so i replace ',' with '.'
pro2['Shipping Weight'] = pro2['Shipping Weight'].str.replace(',', '.')

# Turn all value to pounds (the index with ounces will be devided by 16)
for i, data in pro2['Shipping Weight'].items():
    if 'pound' in data:
        pro2.loc[i, 'Shipping Weight'] = float(data.split()[0])
    elif 'ounce' in data:
        pro2.loc[i, 'Shipping Weight'] = float(data.split()[0])/16
    else:
        pro2.loc[i, 'Shipping Weight'] = pro2.loc[i-1, 'Selling Price']

# Check for Duplicated
pro2.duplicated().sum()

# Change value from 'Subscription Status' column to one format
pro2['Is Amazon Seller'] = pro2['Is Amazon Seller'].str.replace('Y', 'Yes')
pro2['Is Amazon Seller'] = pro2['Is Amazon Seller'].str.replace('N', 'No')

# Convert all data to suitable type
pro2['Selling Price'] = pro2['Selling Price'].astype('float')
pro2['Shipping Weight'] = pro2['Shipping Weight'].astype('float')
pro2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Uniqe Id          10002 non-null  object 
 1   Product Name      10002 non-null  object 
 2   Selling Price     10002 non-null  float64
 3   Shipping Weight   10002 non-null  float64
 4   Is Amazon Seller  10002 non-null  object 
dtypes: float64(2), object(3)
memory usage: 390.8+ KB


In [4]:
# Cleaning
# 2. Customer Data
cus2 = cus.copy()

# Check for Null value
cus2.isnull().sum()

# Check for Duplicated value
cus2.duplicated().sum()
cus2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        3900 non-null   object
 1   Age       3900 non-null   int64 
 2   Gender    3900 non-null   object
 3   Location  3900 non-null   object
dtypes: int64(1), object(3)
memory usage: 122.0+ KB


In [5]:
# Cleaning
# 3. Order Data
ord2 = ord.copy()

# Check for Null value and fill with the above value
ord2.isnull()
ord2['Subscription Status'] = ord2['Subscription Status'].ffill()
ord2['Shipping Type'] = ord2['Shipping Type'].ffill()
ord2['Discount'] = ord2['Discount'].ffill()

# Change value from 'Subscription Status' column to one format
for i, data in ord2['Subscription Status'].items():
    if data == 'Y':
        ord2.loc[i, 'Subscription Status'] = 'Yes'
    elif data == 'Ye':
        ord2.loc[i, 'Subscription Status'] = 'Yes'
    elif data == 'Yess':
        ord2.loc[i, 'Subscription Status'] = 'Yes'
    elif data == 'no':
        ord2.loc[i, 'Subscription Status'] = 'No'
    else: ord2.loc[i, 'Subscription Status'] = data

# Remove % symbol from 'Discount' column
ord2['Discount'] = ord2['Discount'].str.replace('%', '')

# Convert all data to suitable type
ord2['Date'] = ord2['Date'].astype('datetime64[ns]')
ord2['Discount'] = (ord2['Discount'].astype('float'))/100

ord2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12137 entries, 0 to 12136
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Customer ID          12137 non-null  object        
 1   Product ID           12137 non-null  object        
 2   Order ID             12137 non-null  object        
 3   Order Quantity       12137 non-null  int64         
 4   Review Rating        12137 non-null  float64       
 5   Payment Method       12137 non-null  object        
 6   Subscription Status  12137 non-null  object        
 7   Shipping Type        12137 non-null  object        
 8   Discount             12137 non-null  float64       
 9   Date                 12137 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 948.3+ KB


In [19]:
# Cleaning
# Save the cleaned data as 'c_'
c_cus = cus2.copy()
c_cat = cat.copy()
c_pro = pro2.copy()
c_ord = ord2.copy()

# Rename the columns to show similarity across tables
c_pro = c_pro.rename(columns = {'Uniqe Id' : 'Product ID'})
c_cus = c_cus.rename(columns = {'ID' : 'Customer ID'})
c_cat = c_cat.rename(columns = {'Uniqe Id' : 'Product ID'})

c_ord.to_csv('D:\Porfolio\Project 3 - E-Commerce - Cleaned\Cleaned Data\Order Details.csv', index=False)
c_cus.to_csv('D:\Porfolio\Project 3 - E-Commerce - Cleaned\Cleaned Data\Cutomer Details.csv', index=False)
c_cat.to_csv('D:\Porfolio\Project 3 - E-Commerce - Cleaned\Cleaned Data\Category Details.csv', index=False)
c_pro.to_csv('D:\Porfolio\Project 3 - E-Commerce - Cleaned\Cleaned Data\Product Details.csv', index=False)
loca.to_csv('D:\Porfolio\Project 3 - E-Commerce - Cleaned\Cleaned Data\Location Group.csv', index=False)
c_ord_cat.to_csv('D:\Porfolio\Project 3 - E-Commerce - Cleaned\Cleaned Data\Order Category.csv', index=False)

In [7]:
# Join the table for calculation
c_ord_pro = pd.merge(c_ord, c_pro, on='Product ID')
c_ord_cat = pd.merge(c_ord, c_cat, on='Product ID')
c_ord_cus = pd.merge(c_ord, c_cus, on='Customer ID')

In [8]:
# Data metrics

# 1. Total Order
total_order = c_ord['Order ID'].nunique()

# 2. Unique Customer
unique_customer = c_ord['Customer ID'].nunique()

# 3. Unique Product
unique_product = c_ord['Product ID'].nunique()

# 4. Total Item Order
total_item_order = c_ord['Order Quantity'].sum()

# 5. Total Revenue
c_ord_pro['Payment Value'] = c_ord_pro['Order Quantity'] * c_ord_pro['Selling Price'] * c_ord_pro['Discount']
total_revenue = c_ord_pro['Payment Value'].sum().round(decimals=2)

# 6. Average Order Value
average_order_value = (total_revenue / total_order).round(decimals=2)

# 7. Item Ordered per Customer
item_ordered_pre_customer = (total_item_order / unique_customer).round(decimals=2)

# 8. Average Review Score
average_review_score = c_ord['Review Rating'].mean().round(decimals=2)

# 9. Average Discount Value
average_discount = c_ord['Discount'].mean().round(decimals=2)*100
average_discount_value = f"{average_discount}%"

In [9]:
Order_Metrics = pd.DataFrame({'Metrics':[
    total_order,
    unique_customer,
    unique_product,
    total_item_order,
    total_revenue,
    average_order_value,
    item_ordered_pre_customer,
    average_review_score,
    average_discount_value
    ]},index=['Total Order', 
              'Unique Customers',
              'Unique Product',
              'Total Item Ordered',
              'Total Revenue',
              'Average Order Value',
              'Item Ordered per Customer',
              'Average Review Score',
              'Average Discount Value' ])
Order_Metrics.round(decimals=2)

,Metrics
Total Order,12137
Unique Customers,3723
Unique Product,6983
Total Item Ordered,94885
Total Revenue,666578.16
Average Order Value,54.92
Item Ordered per Customer,25.49
Average Review Score,3.75
Average Discount Value,18.0%


In [10]:
#10 Number of Orders over Time
order_year = c_ord['Date'].dt.year
order_month = c_ord['Date'].dt.month

orders_by_time = c_ord.groupby([order_year,order_month])['Order ID'].count()
orders_by_time

Date  Date
2019  1        90
      2        92
      3       106
      4        97
      5        91
      6        85
      7        86
      8        92
      9       112
      10      129
      11      150
      12      133
2020  1       207
      2       225
      3       246
      4       237
      5       210
      6       207
      7       198
      8       225
      9       273
      10      300
      11      351
      12      321
2021  1       303
      2       310
      3       295
      4       269
      5       254
      6       228
      7       292
      8       292
      9       333
      10      363
      11      407
      12      393
2022  1       306
      2       357
      3       352
      4       286
      5       232
      6       261
      7       281
      8       364
      9       430
      10      459
      11      426
      12      381
Name: Order ID, dtype: int64

In [21]:
# 11. Most Ordered Category
c_ord_cat['Category'].value_counts().head(10)

Category
Toys & Games                                8091
Hobbies                                     1165
Home & Kitchen                               853
Games & Accessories                          792
Clothing, Shoes & Jewelry                    758
Costumes                                     670
Arts & Crafts                                658
Costumes & Accessories                       651
Sports & Outdoors                            632
Remote & App Controlled Vehicles & Parts     611
Name: count, dtype: int64

In [18]:
# 12.Number of Orders from each States
loca = c_ord_cus['Location'].value_counts()
loca

Location
Montana           319
Illinois          296
California        285
Minnesota         284
Vermont           268
Georgia           267
Alabama           267
Idaho             266
Mississippi       266
Maryland          264
Nevada            264
New York          264
Oklahoma          263
Delaware          262
Maine             259
Kentucky          259
Missouri          258
Nebraska          257
New Mexico        256
Louisiana         256
West Virginia     254
Ohio              247
Arkansas          246
Indiana           245
North Dakota      245
South Carolina    244
Wyoming           242
Tennessee         241
Utah              239
New Hampshire     238
Iowa              237
Oregon            235
Michigan          233
Virginia          233
Colorado          231
South Dakota      229
Wisconsin         226
Pennsylvania      222
Hawaii            219
New Jersey        217
Texas             216
Alaska            212
Florida           212
Connecticut       209
Arizona           208
N

In [17]:
# Specify the data to "Groups" that can be further used in Power BI
loca = c_ord_cus['Location'].value_counts().reset_index()
loca.columns = ['Location', 'Count']

for i, data in loca['Count'].items():
    if data < 190:
        loca.loc[i, 'Count'] = '< 190'
    elif data >= 190 and data < 210:
        loca.loc[i, 'Count'] = '190 - 210'
    elif data >= 210 and data < 230:
        loca.loc[i, 'Count'] = '210 - 230'
    elif data >= 230 and data < 250:
        loca.loc[i, 'Count'] = '230 - 250'
    elif data >= 250 and data < 270:
        loca.loc[i, 'Count'] = '250 - 270'
    elif data >= 270 and data < 290:
        loca.loc[i, 'Count'] = '270 - 290'
    else: loca.loc[i, 'Count'] = '>290'

print(loca)

          Location      Count
0          Montana       >290
1         Illinois       >290
2       California  270 - 290
3        Minnesota  270 - 290
4          Vermont  250 - 270
5          Georgia  250 - 270
6          Alabama  250 - 270
7            Idaho  250 - 270
8      Mississippi  250 - 270
9         Maryland  250 - 270
10          Nevada  250 - 270
11        New York  250 - 270
12        Oklahoma  250 - 270
13        Delaware  250 - 270
14           Maine  250 - 270
15        Kentucky  250 - 270
16        Missouri  250 - 270
17        Nebraska  250 - 270
18      New Mexico  250 - 270
19       Louisiana  250 - 270
20   West Virginia  250 - 270
21            Ohio  230 - 250
22        Arkansas  230 - 250
23         Indiana  230 - 250
24    North Dakota  230 - 250
25  South Carolina  230 - 250
26         Wyoming  230 - 250
27       Tennessee  230 - 250
28            Utah  230 - 250
29   New Hampshire  230 - 250
30            Iowa  230 - 250
31          Oregon  230 - 250
32        

C:\Users\Windows 10\AppData\Local\Temp\ipykernel_9140\427976629.py:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '>290' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  else: loca.loc[i, 'Count'] = '>290'
